## Getting first and last values

Let us see how we can get first and last value based on the criteria.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
val username = System.getProperty("user.name")

username = itv002480


itv002480

In [2]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Windowing Functions").
    master("yarn").
    getOrCreate

username = itv002480
spark = org.apache.spark.sql.SparkSession@58410f3c


org.apache.spark.sql.SparkSession@58410f3c

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

Here is the example of using first_value.

In [3]:
%%sql

USE itv002480_retail

Waiting for a Spark session to start...

++
||
++
++



In [4]:
%%sql

SELECT t.*,
  first_value(order_item_product_id) OVER (
    PARTITION BY order_date ORDER BY revenue DESC
  ) first_product_id,
  first_value(revenue) OVER (
    PARTITION BY order_date ORDER BY revenue DESC
  ) first_revenue
FROM daily_product_revenue t
ORDER BY order_date, revenue DESC
LIMIT 100

|2013-07-25 00:00:...|                 1014|2...


+--------------------+---------------------+-------+----------------+-------------+
|          order_date|order_item_product_id|revenue|first_product_id|first_revenue|
+--------------------+---------------------+-------+----------------+-------------+
|2013-07-25 00:00:...|                 1004|5599.72|            1004|      5599.72|
|2013-07-25 00:00:...|                  191|5099.49|            1004|      5599.72|
|2013-07-25 00:00:...|                  957| 4499.7|            1004|      5599.72|
|2013-07-25 00:00:...|                  365|3359.44|            1004|      5599.72|
|2013-07-25 00:00:...|                 1073|2999.85|            1004|      5599.72|
|2013-07-25 00:00:...|                 1014|2798.88|            1004|      5599.72|
|2013-07-25 00:00:...|                  403|1949.85|            1004|      5599.72|
|2013-07-25 00:00:...|                  502| 1650.0|            1004|      5599.72|
|2013-07-25 00:00:...|                  627|1079.73|            1004|      5

In [5]:
spark.sql("""
    SELECT t.*,
      first_value(order_item_product_id) OVER (
        PARTITION BY order_date ORDER BY revenue DESC
      ) first_product_id,
      first_value(revenue) OVER (
        PARTITION BY order_date ORDER BY revenue DESC
      ) first_revenue
    FROM daily_product_revenue t
    ORDER BY order_date, revenue DESC
""").
    show(100, false)

+---------------------+---------------------+--------+----------------+-------------+
|order_date           |order_item_product_id|revenue |first_product_id|first_revenue|
+---------------------+---------------------+--------+----------------+-------------+
|2013-07-25 00:00:00.0|1004                 |5599.72 |1004            |5599.72      |
|2013-07-25 00:00:00.0|191                  |5099.49 |1004            |5599.72      |
|2013-07-25 00:00:00.0|957                  |4499.7  |1004            |5599.72      |
|2013-07-25 00:00:00.0|365                  |3359.44 |1004            |5599.72      |
|2013-07-25 00:00:00.0|1073                 |2999.85 |1004            |5599.72      |
|2013-07-25 00:00:00.0|1014                 |2798.88 |1004            |5599.72      |
|2013-07-25 00:00:00.0|403                  |1949.85 |1004            |5599.72      |
|2013-07-25 00:00:00.0|502                  |1650.0  |1004            |5599.72      |
|2013-07-25 00:00:00.0|627                  |1079.73 |

Let us see an example with last_value. While using last_value we need to specify **ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING**.
* By default it uses `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`.
* The last value with in `UNBOUNDED PRECEDING AND CURRENT ROW` will be current record.
* To get the right value, we have to change the windowing clause to `ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING`.

In [6]:
%%sql

USE itv002480_retail

++
||
++
++



In [ ]:
%%sql

SELECT t.*,
    last_value(order_item_product_id) OVER (
        PARTITION BY order_date ORDER BY revenue
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) last_product_id,
    last_value(revenue) OVER (
        PARTITION BY order_date ORDER BY revenue
    ) last_revenue
FROM daily_product_revenue AS t
ORDER BY order_date, revenue DESC
LIMIT 100

In [ ]:
%%sql

SELECT t.*,
    last_value(order_item_product_id) OVER (
        PARTITION BY order_date ORDER BY revenue
        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING
    ) last_product_id,
    last_value(revenue) OVER (
        PARTITION BY order_date ORDER BY revenue
        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING
    ) last_revenue
FROM daily_product_revenue AS t
ORDER BY order_date, revenue DESC
LIMIT 100

In [ ]:
spark.sql("""
    SELECT t.*,
      last_value(order_item_product_id) OVER (
        PARTITION BY order_date ORDER BY revenue
        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING
      ) last_product_id,
      last_value(revenue) OVER (
        PARTITION BY order_date ORDER BY revenue
        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING
      )  last_revenue
    FROM daily_product_revenue AS t
    ORDER BY order_date, revenue DESC
""").
    show(100, false)